In [ ]:
import datetime
from zoneinfo import ZoneInfo
from google.adk.agents import Agent, SequentialAgent
from google.adk.models.lite_llm import LiteLlm
from google.adk.tools import google_search

In [4]:
#AGENT_MODEL = "ollama/gemma3"
# AGENT_MODEL = "openai/gpt-5-nano"
AGENT_MODEL = "gemini-2.5-flash"


def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city (e.g., "New York", "London", "Tokyo").

    Returns:
        dict: A dictionary containing the weather information.
              Includes a 'status' key ('success' or 'error').
              If 'success', includes a 'report' key with weather details.
              If 'error', includes an 'error_message' key.
    """
    print(f"--- Tool: get_weather called for city: {city} ---")  # Log tool execution
    city_normalized = city.lower().replace(" ", "")  # Basic normalization

    # Mock weather data
    # api call
    mock_weather_db = {
        "newyork": {
            "status": "success",
            "report": "The weather in New York is sunny with a temperature of 25°C.",
        },
        "london": {
            "status": "success",
            "report": "It's cloudy in London with a temperature of 15°C.",
        },
        "tokyo": {
            "status": "success",
            "report": "Tokyo is experiencing light rain and a temperature of 18°C.",
        },
        "paris": {
            "status": "success",
            "report": "The weather in Paris is sunny with a temperature of 22°C.",
        },
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return {
            "status": "error",
            "error_message": f"Sorry, I don't have weather information for '{city}'.",
        }


def get_current_time(city: str) -> dict:
    """Returns the current time in a specified city.

    Args:
        city (str): The name of the city for which to retrieve the current time.

    Returns:
        dict: status and result or error msg.
    """

    if city.lower() == "new york":
        tz_identifier = "America/New_York"
    else:
        return {
            "status": "error",
            "error_message": (f"Sorry, I don't have timezone information for {city}."),
        }

    tz = ZoneInfo(tz_identifier)
    now = datetime.datetime.now(tz)
    report = f'The current time in {city} is {now.strftime("%Y-%m-%d %H:%M:%S %Z%z")}'
    return {"status": "success", "report": report}


# -- Sequential Agent ---
# Destination Research Agent - Researches location information
destination_research_agent = Agent(
    name="DestinationResearchAgent",
    model="gemini-2.5-flash",
    tools=[google_search],
    description="An agent that researches travel destinations and gathers essential information",
    instruction="""
    You are a travel researcher. You will be given a destination and travel preferences, and you will research:
    - Best time to visit and weather patterns
    - Top attractions and must-see locations
    - Local culture, customs, and etiquette tips
    - Transportation options within the destination
    - Safety considerations and travel requirements
    Provide comprehensive destination insights for trip planning.
    """,
    output_key="destination_research",
)


# Itinerary Builder Agent - Creates detailed travel schedule
itinerary_builder_agent = Agent(
    model=AGENT_MODEL,
    name="ItineraryBuilderAgent",
    description="An agent that creates structured travel itineraries with daily schedules",
    instruction="""
    You are a professional travel planner. Using the research from "destination_research" output, create a detailed itinerary that includes:
    - Day-by-day schedule with recommended activities
    - Suggested accommodation areas or districts
    - Estimated time requirements for each activity
    - Meal recommendations and dining suggestions
    - Budget estimates for major expenses
    Structure it logically for easy following during the trip.
    """,
    output_key="travel_itinerary",
)

# Travel Optimizer Agent - Adds practical tips and optimizations
travel_optimizer_agent = Agent(
    model=AGENT_MODEL,
    name="TravelOptimizerAgent",
    description="An agent that optimizes travel plans with practical advice and alternatives",
    instruction="""
    You are a seasoned travel consultant. Using the itinerary from "travel_itinerary" output, optimize it by adding:
    - Money-saving tips and budget alternatives
    - Packing recommendations specific to the destination
    - Backup plans for weather or unexpected situations
    - Local apps, websites, or resources to download
    - Cultural do's and don'ts for respectful travel
    
    Format the final output as:
    
    ITINERARY: {travel_itinerary}
    
    OPTIMIZATION TIPS: [your money-saving and practical tips here]
    
    TRAVEL ESSENTIALS: [packing and preparation advice here]
    
    BACKUP PLANS: [alternative options and contingencies here]
    """,
)

root_agent = SequentialAgent(
    name="TravelPlanningSystem",
    # model=LiteLlm(AGENT_MODEL),#not needed for SequentialAgent
    # model=AGENT_MODEL, #not needed for SequentialAgent
    description="A comprehensive system that researches destinations, builds itineraries, and optimizes travel plans",
    sub_agents=[
        destination_research_agent,
        itinerary_builder_agent,
        travel_optimizer_agent,
    ],
    # instruction="You are a travel planner agent. Help the user plan their trip.",
    # tools=[get_weather, get_current_time],
)

In [8]:
project_id = 'agent'

In [14]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
from pathlib import Path
from typing import Optional

# Load environment variables (works in both notebooks and scripts)
env_path = Path.cwd() / ".env"  # Use current working directory instead of __file__
if env_path.exists():
    load_dotenv(dotenv_path=str(env_path), override=True)
else:
    # Try parent directory if .env is not in current directory
    env_path = Path.cwd().parent / ".env"
    if env_path.exists():
        load_dotenv(dotenv_path=str(env_path), override=True)
    else:
        # Fallback: just load from current directory
        load_dotenv(override=False)


def generate_prompt_with_gemini(
    user_input: str,
    model: str = "gemini-2.5-flash",
    temperature: float = 0.7,
    max_output_tokens: Optional[int] = None,
    system_instruction: Optional[str] = None,
    project_id: Optional[str] = None,
    location: str = "us-central1"
) -> str:
    """
    Generate a prompt or text response using Google's Gemini model via Vertex AI.
    
    Args:
        user_input (str): The input prompt or question to send to Gemini
        model (str): The Gemini model to use (default: "gemini-2.0-flash")
                    Options: "gemini-2.0-flash", "gemini-1.5-flash", "gemini-1.5-pro", etc.
                    Note: Vertex AI model names typically end with "-001" (e.g., "gemini-2.0-flash-001")
        temperature (float): Controls randomness (0.0-1.0). Higher = more creative (default: 0.7)
        max_output_tokens (Optional[int]): Maximum tokens in response (None = model default)
        system_instruction (Optional[str]): Optional system instruction to set the model's behavior
        project_id (Optional[str]): Google Cloud project ID. If None, uses GOOGLE_CLOUD_PROJECT env var
        location (str): Google Cloud location/region (default: "us-central1")
    
    Returns:
        str: The generated text response from Gemini
    
    Example:
        >>> response = generate_prompt_with_gemini("Create a travel itinerary for Tokyo")
        >>> print(response)
    """
    # Get API key from environment
    google_api_key = os.getenv("GOOGLE_API_KEY")
    if not google_api_key:
        raise ValueError("GOOGLE_API_KEY not found in environment variables")
    
    # Configure the API
    genai.configure(api_key=google_api_key)
    
    # Configure generation parameters
    generation_config = {
        "temperature": temperature,
    }
    if max_output_tokens:
        generation_config["max_output_tokens"] = max_output_tokens
    
    # Initialize the model
    model_instance = genai.GenerativeModel(
        model_name=model,
        system_instruction=system_instruction,
        generation_config=generation_config
    )
    
    # Generate response
    response = model_instance.generate_content(user_input)
    
    # Extract text from the response
    return response.text




c:\Users\wsk2000\Documents\Github_codes\personal_assistant\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
style_instructions = {
        "general": "You are a helpful assistant.",
        "creative": "You are a creative writer. Be imaginative and engaging.",
        "technical": "You are a technical expert. Be precise and detailed.",
        "concise": "You are a concise assistant. Be brief and to the point."
    }
    
system_instruction = style_instructions["general"]

In [12]:
full_prompt = 'what is the capital of the moon?'

In [15]:
result = generate_prompt_with_gemini(
        full_prompt,
        system_instruction=system_instruction,
        model="gemini-2.5-flash",  
        project_id=project_id
    )

In [16]:
result

"The Moon does not have a capital.\n\nIt's not a country or a political entity, so it doesn't have cities, governments, or anything like that."